In [1]:
!nvidia-smi

Mon Jan 30 22:25:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   58C    P8    17W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!python --version

Python 3.8.13


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
import torch 
torch.__version__

'1.13.1+cu117'

In [5]:
import transformers
transformers.__version__

'4.22.2'

In [6]:
from joblib import Parallel, delayed
from math import sqrt
Parallel(n_jobs=2)(delayed(sqrt)(i ** 2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

# unoptimized onnx

In [8]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM


In [9]:
tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")


In [10]:
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=False, provider='CUDAExecutionProvider')


2023-01-30 22:29:36.400291409 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-30 22:29:36.400327604 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-30 22:29:52.504735888 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-30 22:29:52.504771001 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-30 22:30:07.758091475 [W:onnxrun

In [11]:
!nvidia-smi

Mon Jan 30 22:30:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   45C    P0    34W /  70W |   6417MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
model_ort2 = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=False, provider='CUDAExecutionProvider')

2023-01-30 22:30:11.589494769 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-30 22:30:11.589534728 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-30 22:30:15.121773570 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-30 22:30:15.121797040 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-30 22:30:21.055297545 [W:onnxrun

In [13]:
!nvidia-smi

Mon Jan 30 22:30:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   46C    P0    34W /  70W |  11791MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## repeat all combined test parallel

In [28]:
def inf_query(query, use_beam, session, repeat=1):
    for _ in range(repeat):
        inputs = tokenizer(f"Generate taxonomy for query: {query}", return_tensors="pt").to('cuda')
        if use_beam:
            outputs = session.generate(**inputs, num_beams=3, num_return_sequences=3)
        else:
            outputs = session.generate(**inputs)
        res = tokenizer.batch_decode(outputs)
    return res

In [29]:
from functools import partial

In [30]:
inf_query(query='dildo', use_beam=False, session=model_ort)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']

In [31]:
partial(inf_query, query='dildo', use_beam=False)(session=model_ort)

['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']

In [32]:
%%timeit
Parallel(n_jobs=2, backend='threading')(delayed(partial(inf_query, query='dildo', use_beam=False, repeat=10))(session=i) for i in [model_ort, model_ort2])

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


2.53 s ± 32.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
partial(inf_query, query='dildo', use_beam=False, repeat=10)(session=model_ort)

1.42 s ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
2.53 / 1.42

1.7816901408450703